In [5]:
# !pip3 install -q langchain==0.0.152   

In [1]:
# !pip3 install -q pyllamacpp==1.0.7    

In [3]:
# !pip install -q sentencepiece

In [4]:
import os
import json 
from dotenv import load_dotenv
load_dotenv('./.env')

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
ACTIVELOOP_TOKEN = os.getenv('ACTIVELOOP_TOKEN')
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
GOOGLE_CSE_ID = os.getenv('GOOGLE_CSE_ID')
HUGGINGFACEHUB_API_TOKEN  = os.getenv('HUGGINGFACEHUB_API_TOKEN')

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["ACTIVELOOP_TOKEN"] = ACTIVELOOP_TOKEN
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["GOOGLE_CSE_ID"] = GOOGLE_CSE_ID
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [7]:
local_path = r'C:\Users\ElshanChalabiyev\Desktop\Codes_for_Insights\Activeloop-Course\2.6. Using_the_Open_Source_GPT4All_Model_Locally.ipynb\models/gpt4all-lora-quantized-ggml.bin'  # replace with your desired local file path

In [8]:
import requests
from pathlib import Path
from tqdm import tqdm

local_path = './models/gpt4all-lora-quantized-ggml.bin'
Path(local_path).parent.mkdir(parents=True, exist_ok=True)

# Example model. Check https://github.com/nomic-ai/pyllamacpp for the latest models.
url = 'https://the-eye.eu/public/AI/models/nomic-ai/gpt4all/gpt4all-lora-quantized-ggml.bin'
# url = 'http://gpt4all.io/models/ggml-gpt4all-l13b-snoozy.bin'

# send a GET request to the URL to download the file. Stream since it's large
response = requests.get(url, stream=True)

# open the file in binary mode and write the contents of the response to it in chunks
# This is a large file, so be prepared to wait.
with open(local_path, 'wb') as f:
    for chunk in tqdm(response.iter_content(chunk_size=8192)):
        if chunk:
            f.write(chunk)

514266it [08:30, 1008.20it/s]


https://github.com/ggerganov/llama.cpp#using-gpt4all

In [9]:
!git clone https://github.com/ggerganov/llama.cpp.git

Cloning into 'llama.cpp'...


In [10]:
!python3 llama.cpp/convert.py ./models/gpt4all-lora-quantized-ggml.bin

Loading model file models\gpt4all-lora-quantized-ggml.bin
params: n_vocab:32001 n_embd:4096 n_mult:256 n_head:32 n_layer:32
Writing vocab...
[  1/291] Writing tensor tok_embeddings.weight                  | size  32001 x   4096  | type QuantizedDataType(groupsize=32, have_addends=False, have_g_idx=False)
[  2/291] Writing tensor norm.weight                            | size   4096           | type UnquantizedDataType(name='F32')
[  3/291] Writing tensor output.weight                          | size  32001 x   4096  | type QuantizedDataType(groupsize=32, have_addends=False, have_g_idx=False)
[  4/291] Writing tensor layers.0.attention.wq.weight           | size   4096 x   4096  | type QuantizedDataType(groupsize=32, have_addends=False, have_g_idx=False)
[  5/291] Writing tensor layers.0.attention.wk.weight           | size   4096 x   4096  | type QuantizedDataType(groupsize=32, have_addends=False, have_g_idx=False)
[  6/291] Writing tensor layers.0.attention.wv.weight           | size  

In [11]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import GPT4All
from langchain.callbacks.base import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [12]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [13]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [14]:
# Verbose is required to pass to the callback manager
llm = GPT4All(model="./models/ggml-model-q4_0.bin", callback_manager=callback_manager, verbose=True)

In [15]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

# Documentation Example (!?)

In [16]:
question = "What NFL team won the Super Bowl in the year Justin Bieber was born?"

In [17]:
llm_chain.run(question)

 Question: What NFL team won the Super Bowl in the year Justin Bieber was born?

Answer: Let's think step by step. First, to answer this question we need some basic information such as when and where Justin Bieber (JB) was actually conceived or created. This would have happened sometime between 1986-2005 since that is the range of years JB has said he began performing in public with his family band, The Band Perry playing instruments like piano keyboards as well and violin!

" Question: What NFL team won the Super Bowl in the year Justin Bieber was born?\n\nAnswer: Let's think step by step. First, to answer this question we need some basic information such as when and where Justin Bieber (JB) was actually conceived or created. This would have happened sometime between 1986-2005 since that is the range of years JB has said he began performing in public with his family band, The Band Perry playing instruments like piano keyboards as well and violin!"

# Poetic Example

In [18]:
question = "Write a poem about friendship that rhymes."

In [19]:
llm_chain.run(question)

 Question: Write a poem about friendship that rhymes.

Answer: Let's think step by step. Friendship is the perfect union of two souls who are inseparable as time goes on, but like any relationship it needs maintenance and attention for survival through thick or thin. So here I go with my poem 'Friendship': A poem about a lifelong friendship that rhymes! 
When you meet someone special at the very beginning of your life journey - there's no telling where this path might lead us in time, when we met each other on our way to becoming friends. We were just two strangers who became acquainted and found it easy to talk with one another, but as they say "The Rest is History"! 
You may ask yourself how long can a relationship last? Well I don't have the answer for that -but what I do know is this: A true friend will stay by your side no matter where you go or whatever happens in life. The thing about our friendship, it has grown and bloomed into something more than just being friends! 
Now let 

' Question: Write a poem about friendship that rhymes.\n\nAnswer: Let\'s think step by step. Friendship is the perfect union of two souls who are inseparable as time goes on, but like any relationship it needs maintenance and attention for survival through thick or thin. So here I go with my poem \'Friendship\': A poem about a lifelong friendship that rhymes! \nWhen you meet someone special at the very beginning of your life journey - there\'s no telling where this path might lead us in time, when we met each other on our way to becoming friends. We were just two strangers who became acquainted and found it easy to talk with one another, but as they say "The Rest is History"! \nYou may ask yourself how long can a relationship last? Well I don\'t have the answer for that -but what I do know is this: A true friend will stay by your side no matter where you go or whatever happens in life. The thing about our friendship, it has grown and bloomed into something more than just being friends!

# Mother's day Example

In [ ]:
question = "Write a social media post to celebrate mother's day."

In [ ]:
llm_chain.run(question)

 Question: Write a social media post to celebrate mother's day.

Answer: Let's think step by step. First, we need an attractive and catchy headline that will make your mom feel appreciated on this special occasion! Here are some examples of what you could write as the heading:"Mommy, You Are The Best Thing That Has Ever Happened To Me" or "I Am So Lucky to Have a Mother Like YOU".
Now it's time for an introduction. Write about your mom’s accomplishments and achievements that make her special:“My mother has the most contagious laugh I have ever heard, she can turn any ordinary day into a happy one.” or “Mommy, You are always there to listen when we need you”
Then comes the body of your post. This is where you'll share all those adorable childhood memories with her and show how much they mean to both of you:“My mom has taught me so many things that I would never have learned otherwise.” or “I can’t imagine a day without my Mommy”
In conclusion, write something heartfelt about your mother

' Question: Write a social media post to celebrate mother\'s day.\n\nAnswer: Let\'s think step by step. First, we need an attractive and catchy headline that will make your mom feel appreciated on this special occasion! Here are some examples of what you could write as the heading:"Mommy, You Are The Best Thing That Has Ever Happened To Me" or "I Am So Lucky to Have a Mother Like YOU".\nNow it\'s time for an introduction. Write about your mom’s accomplishments and achievements that make her special:“My mother has the most contagious laugh I have ever heard, she can turn any ordinary day into a happy one.” or “Mommy, You are always there to listen when we need you”\nThen comes the body of your post. This is where you\'ll share all those adorable childhood memories with her and show how much they mean to both of you:“My mom has taught me so many things that I would never have learned otherwise.” or “I can’t imagine a day without my Mommy”\nIn conclusion, write something heartfelt about y

# Explain Rain Example

In [ ]:
question = "What happens when it rains somewhere?"

In [ ]:
llm_chain.run(question)

 Question: What happens when it rains somewhere?

Answer: Let's think step by step. When rain falls, first of all the water vaporizes from clouds and travel to lower altitude where air is denser. Then these drops hit surfaces like land or trees etc., which are considered as a target for this falling particles known as rainfall. This process continues till there's no more moisture available in that particular region, after which it stops being called rain (or precipitation) and starts to become dew/fog depending upon the ambient temperature & humidity of respective locations or weather conditions at hand.

" Question: What happens when it rains somewhere?\n\nAnswer: Let's think step by step. When rain falls, first of all the water vaporizes from clouds and travel to lower altitude where air is denser. Then these drops hit surfaces like land or trees etc., which are considered as a target for this falling particles known as rainfall. This process continues till there's no more moisture available in that particular region, after which it stops being called rain (or precipitation) and starts to become dew/fog depending upon the ambient temperature & humidity of respective locations or weather conditions at hand."

# Rain Example, but one sentence and funny.

In [ ]:
template = """Question: {question}

Answer: Let's answer in two sentence while being funny."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [ ]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
llm_chain.run(question)

 Question: What happens when it rains somewhere?

Answer: Let's answer in two sentence while being funny. 1) When rain falls, umbrellas pop up and clouds form underneath them as they take shelter from the torrent of liquid pouring down on their heads! And...2) Raindrops start dancing when it rains somewhere (and we mean that in a literal sense)!

" Question: What happens when it rains somewhere?\n\nAnswer: Let's answer in two sentence while being funny. 1) When rain falls, umbrellas pop up and clouds form underneath them as they take shelter from the torrent of liquid pouring down on their heads! And...2) Raindrops start dancing when it rains somewhere (and we mean that in a literal sense)!"